In [344]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
onestop = re.get('https://www.onestopautogarage.co.ke/')
kmautos = re.get('https://kmautospares.com/shop/')
xpress = re.get('https://auto-xpress.co.ke/estore/')
amex = re.get('https://amexautoparts.com/?srsltid=AfmBOoqsbMFLN5yOZmAFuzCVeglm2r19QATiIbS-b6_FaI27CE7MSxKt')
jiji = re.get('https://jiji.co.ke/car-parts-and-accessories')
tyrex = re.get('https://www.tyrexkenya.co.ke/')

In [ ]:

jiji_container = BeautifulSoup(jiji.content,'html.parser')
tyrex_container = BeautifulSoup(tyrex.content,'html.parser')

## One stop Website

In [ ]:
Product_names = []
Product_prices = []
Outlet = []
Website = []
Original_prices = []
onestop_links = []

onestop_container = BeautifulSoup(onestop.content,'html.parser')
onestop_Category = onestop_container.find_all('div', {'class':'category-image-wrapp'})
for link in range(len(onestop_Category)):
    onestop_links.append(onestop_Category[link].find('a')['href'])

for a in range(len(onestop_links)):
    productload = re.get(onestop_links[a])
    products = BeautifulSoup(productload.content,'html.parser')
    product_items =products.find_all('div', {'class':'product-wrapper'})
    for i in range(len(product_items)):
        Product_names.append(product_items[i].find('h3').text)
        Outlet.append('One Stop Auto Garage')
        try:
            Product_prices.append(product_items[i].find('span', {'class':'price'}).text.split('.00')[2].replace(".KSh\xa0",""))
            Original_prices.append(product_items[i].find('span', {'class':'price'}).text.split('.00 Original')[0].replace("KSh",""))
        except AttributeError:
            Product_prices.append('Not listed')
            Original_prices.append('Not listed')
            i+=1
        Website.append(onestop_links[a].strip('https://www.'))
dataf = pd.DataFrame({'Product Name': Product_names, 'Original price':Original_prices ,'Product Price': Product_prices, 'Outlet': Outlet, 'Link': Website})
dataf.head()

## KMS AUTO Website

In [ ]:
products = []
Websites = []
Outlets = []
Products_name = []
Product_price = []
Old_price = []

BASE_URL = "https://kmautospares.com/shop/"
page_url = BASE_URL

while page_url:

    kmautos = re.get(page_url)
    if kmautos.status_code != 200:
        break
    kmautos_container = BeautifulSoup(kmautos.content,'html.parser')
    product_list = kmautos_container.find_all("li", class_="product")

    for product in product_list:
        name_tag = product.find("h2", class_="woocommerce-loop-product__title")

        # Extract price
        price_tag = product.find("span", class_="price")

        Outlets.append('KM Auto Spares')

        Products_name.append(name_tag.text if name_tag else "N/A")

        Product_price.append(price_tag.text.split(': KSh')[2].replace(".00.","") if price_tag else "N/A")
        Old_price.append(price_tag.text.split('.00 Original')[0].replace("KSh","") if price_tag else "N/A")
        Websites.append(page_url.strip('https://'))

    # Find next page
    next_link = kmautos_container.find("a", class_="next page-numbers")
    page_url = next_link["href"] if next_link else None

# Find the "Next" page URL
next_page = kmautos_container.select_one("a.next.page-numbers")
page_url = next_page['href'] if next_page else None
    
dataf2 = pd.DataFrame({'Product Name': Products_name, 'Original price': Old_price, 'Product Price': Product_price, 'Outlet': Outlets, 'Link': Websites})
dataf2.head()

## Xpress website

In [ ]:
product_namees = []
product_pricees = []
rating = []
Websites = []

xpress_container = BeautifulSoup(xpress.content,'html.parser')
xpress_links = [[xpress_container.find_all('a',{'class':'ct-link-text'})[i]['href'] for i in range(8)][i].replace('services-','') for i in range(len(xpress_links))]

for i in range(len(xpress_links)):
    url = xpress_links[i]
    while url:
        produces = re.get(url)
        product_container = BeautifulSoup(produces.content,'html.parser')
        product_items = product_container.find_all('a', {'class':'box-link'})
        for i in range(len(product_items)):
            product_namees.append(product_items[i].find('div', {'class':'grey-box-loop'}).text)
            product_pricees.append(product_items[i].find('div', {'class':'title-price'}).text.split('KSh\xa0')[1])
            rating.append(product_items[i].find('div', {'class':'title-price'}).text.split(' ')[1])
            Websites.append(url.split('https://auto-xpress.co.ke/')[1])

        # Next Page
        next_page = product_container.find('a',{'class':'wpv-filter-next-link js-wpv-pagination-next-link page-link'})
        if next_page:
            url = url+next_page['href']
        else:
            url = None
            break
    i+=1
Outlets = ['Auto Xpress'] * len(product_namees)

dataf3 = pd.DataFrame({'Product Name': product_namees,'Original price': product_pricees, 'Product Price': product_pricees, 'Outlet':Outlets,'Link':Websites})
dataf3.head()

## Amex Autoparts

In [ ]:
Product_nams = []
Product_priceess = []
Websites = []

amex_container = BeautifulSoup(amex.content,'html.parser')
amex_category_links = [amex_container.find_all('a',{'class':'nav-link dropdown-toggle'})[i]['href']for i in range(len(amex_container.find_all('a',{'class':'nav-link dropdown-toggle'})))]
baseURL = 'https://amexautoparts.com/?srsltid=AfmBOoqsbMFLN5yOZmAFuzCVeglm2r19QATiIbS-b6_FaI27CE7MSxKt'
Webpage = [baseURL+ amex_category_links[i] for i in range(len(amex_category_links))]
for i in range(len(Webpage)):
    nexxts = Webpage[i]
    responses = re.get(nexxts)
    print(f'Scrapping {Webpage[i]}')
    j = 1
    while responses.status_code == 200 and j<216:
        amex_container = BeautifulSoup(responses.content,'html.parser')
        Product_nams.append(amex_container.find_all('h5',{'class':'card-title product-card-name'})[i].text)
        Product_priceess.append(amex_container.find_all('span',{'class':'stockrecord-price-current'})[i].text.split('KES ')[1])
        Websites.append(Webpage[i].split('https://amexautoparts.com/?srsltid=AfmBOoqsbMFLN5yOZmAFuzCVeglm2r19QATiIbS-b6_FaI27CE7MSxKt')[1])
        j += 1
        next_page =Webpage[i] +'?page='+ str(j)
        responses = re.get(next_page)
        nexxts = BeautifulSoup(responses.content,'html.parser')
Outlets = ['Amex Autoparts'] * len(Product_nams)
dataf4 = pd.DataFrame({'Product Name': Product_nams, 'Original price': Product_priceess, 'Product Price': Product_priceess, 'Outlet': Outlets, 'Link': Websites})

In [ ]:
dataf4['Original price'] = [dataf4['Original price'][i].split('\n')[0] for i in range(len(dataf4['Original price']))]
dataf4['Product Price'] = [dataf4['Product Price'][i].split('\n')[0] for i in range(len(dataf4['Product Price']))]

## Combined Master Dataframe

In [ ]:
df = pd.concat([dataf, dataf2,dataf3,dataf4], ignore_index=True)
df.head()

In [ ]:
df.tail(30)